In [4]:
%pip install qiskit
%pip install qiskit_machine_learning
%pip install qiskit_algorithms

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Data management
import numpy as np
import pandas as pd

# Maths
import math
from math import pi

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

# Additional imports
import pylab as pl
from random import random
from numpy import linalg
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

# Plot configuration
%matplotlib inline
sns.set_theme()
sns.set_context("poster")
sns.set_style("ticks")

In [3]:
class Perceptron_Online:
    """
    Perceptron classifier trained using online learning algorithm.

    Parameters:
    -----------
    max_iter : int, optional
        Maximum number of iterations (epochs) for training. Defaults to 10000.
    shuffle : bool, optional
        Whether to shuffle the training dataset between each correction. Defaults to True.
    quantum : bool, optional
        If True, use quantum search for finding misclassified points. Defaults to False.
    nb_ampli : int, optional
        Amplification parameter for quantum search. Defaults to 10.
    opti : bool, optional
        If True, use optimized Grover's algorithm for quantum search. Defaults to False.
    """

    def __init__(self, n_iter=None, shuffle=True, quantum=False, nb_ampli=10, opti=False, gamma=0.01):
        self.shuffle = shuffle  # If we shuffle the training dataset between each correction
        self.coef_ = np.array([0.])  # Default hyperplane
        self.n_iter_ = 0  # Number of iteration (include the number of steps used for the search)
        self.n_correction_ = 0  # Number of correction
        self.quantum = quantum  # If we use the quantum search
        self.nb_ampli = nb_ampli  # The amplification parameter for the quantum search
        self.opti = opti  # If we use the opti Grover
        self.gamma = gamma

    def fit(self, X, y):
        """
        Train the perceptron classifier.

        Parameters:
        -----------
        X : array-like of shape (n_samples, n_features)
            Training data.
        y : array-like of shape (n_samples,)
            Target labels.

        Returns:
        --------
        int
            The number of iterations.

        Updates the model's coefficient during training.
        """
        max_iter = int(len(X)/self.gamma**2)

        b = True
        self.coef_ = np.array([0.] * len(X[0]))  # Initialization of the coef (can be removed if you want to train successively)
        # Copy of the entry (for the shuffle step)
        X_ = np.array([[j for j in i] for i in X])
        y_ = np.array([1 if i == 1 else -1 for i in y])  # Security to ensure that the classes are {-1,1} and not {0,1}.

        nb = 0
        while b:
            nb += 1
            if nb > max_iter:
                break
            b = False

            oracle = [int(y_[i] * X_[i, :].dot(self.coef_) <= 0) for i in range(len(y_))]  # Oracle for the search

            # The right search (according to the options) is used.
            m, steps = classical_search(oracle) if not self.quantum else quantum_search(oracle, nb_ampli=self.nb_ampli, opti=self.opti)
            self.n_iter_ += steps  # We add the number of steps to the model

            if y_[m] * X_[m, :].dot(self.coef_) <= 0:  # If the search is successful we correct
                self.coef_ = self.coef_ + y_[m] * X_[m, :]
                b = True
                self.n_correction_ += 1
                nb += 1

            if self.shuffle:  # Shuffle
                l = list(range(len(X)))
                np.random.shuffle(l)
                X_ = np.array([X[i] for i in l])
                y_ = np.array([1 if y[i] == 1 else -1 for i in l])
        return self.n_iter_

    ##Este método predice las etiquetas de clase para las muestras X dadas.
    def predict(self, X): #X : array-like of shape (n_samples, n_features)  Samples.
        #return array-like of shape (n_samples,) -> Predicted class labels.
        return np.array([1 if x.dot(self.coef_) > 0 else -1 for x in X])